# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234747220702                   -0.50    7.0
  2   -7.250372712776       -1.81       -1.40    1.0
  3   -7.251304931034       -3.03       -2.12    4.0
  4   -7.251294106361   +   -4.97       -2.24    4.0
  5   -7.251335246456       -4.39       -2.87    3.0
  6   -7.251336799091       -5.81       -3.01    2.0
  7   -7.251338779828       -5.70       -4.06    1.0
  8   -7.251338794453       -7.83       -4.27    4.0
  9   -7.251338798586       -8.38       -5.02    3.0
 10   -7.251338798687      -10.00       -5.47    2.0
 11   -7.251338798703      -10.80       -6.00    3.0
 12   -7.251338798704      -11.96       -6.32    2.0
 13   -7.251338798705      -12.68       -6.82    2.0
 14   -7.251338798705      -13.64       -7.14    3.0
 15   -7.251338798705      -14.15       -7.55    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07902036135937103
[ Info: Arnoldi iteration step 2: normres = 0.6651372263686938
[ Info: Arnoldi iteration step 3: normres = 0.686209781336307
[ Info: Arnoldi iteration step 4: normres = 0.2542894245460684
[ Info: Arnoldi iteration step 5: normres = 0.4933675694726326
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.96e-02, 7.27e-02, 2.91e-01, 3.91e-01, 1.59e-02)
[ Info: Arnoldi iteration step 6: normres = 0.21601208572562333
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.78e-03, 1.05e-01, 1.66e-01, 4.67e-02, 2.29e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06878827502433124
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.18e-04, 5.95e-03, 9.04e-03, 1.14e-02, 6.39e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09591702824522959
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (4.84e-0